<a href="https://colab.research.google.com/github/pulum03/ML/blob/master/faster_rcnn_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setting

In [0]:
# pytorch 1.0.0 버전으로 설치하고, 
# git clone 해서 모델  옮기고, 그외 requirements 들 충족하도록 설치 (trello에 있음)

In [1]:
!pip install -q torch==1.0.0 torchvision==0.2.1

     |████████████████████████████████| 591.8MB 29kB/s 
     |████████████████████████████████| 61kB 7.7MB/s 


In [2]:
!git clone -b pytorch-1.0 https://github.com/jwyang/faster-rcnn.pytorch.git

Cloning into 'faster-rcnn.pytorch'...
remote: Enumerating objects: 3846, done.
remote: Total 3846 (delta 0), reused 0 (delta 0), pack-reused 3846
Receiving objects: 100% (3846/3846), 6.21 MiB | 4.15 MiB/s, done.
Resolving deltas: 100% (2608/2608), done.


In [3]:
!pip list

Package                  Version    
------------------------ -----------
absl-py                  0.9.0      
alabaster                0.7.12     
albumentations           0.1.12     
altair                   4.0.0      
asgiref                  3.2.3      
astor                    0.8.1      
astropy                  4.0        
atari-py                 0.2.6      
atomicwrites             1.3.0      
attrs                    19.3.0     
audioread                2.1.8      
autograd                 1.3        
Babel                    2.8.0      
backcall                 0.1.0      
backports.tempfile       1.0        
backports.weakref        1.0.post1  
beautifulsoup4           4.6.3      
bleach                   3.1.0      
blis                     0.2.4      
bokeh                    1.4.0      
boto                     2.49.0     
boto3                    1.10.47    
botocore                 1.13.47    
Bottleneck               1.3.1      
branca                   0.3.1      
b

In [0]:
#!git clone https://github.com/jwyang/faster-rcnn.pytorch.git

fatal: destination path 'faster-rcnn.pytorch' already exists and is not an empty directory.


In [0]:
#!rm -rf faster-rcnn.pytorch

In [0]:
#!git branch

fatal: not a git repository (or any of the parent directories): .git


In [4]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
import torch
print(torch.__version__)

1.0.0


#Data Preparation

In [6]:
!pip install https://github.com/fastai/fastai/archive/master.zip
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6
!mkdir data
!wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar -P data/
!wget https://storage.googleapis.com/coco-dataset/external/PASCAL_VOC.zip -P data/
!tar -xf data/VOCtrainval_06-Nov-2007.tar -C data/
!unzip data/PASCAL_VOC.zip -d data/
!rm -rf data/PASCAL_VOC.zip data/VOCtrainval_06-Nov-2007.tar

     | 133.9MB 35.9MB/s
  Created wheel for fastai: filename=fastai-1.0.61.dev0-cp36-none-any.whl size=237453 sha256=2f497ccdb113ac79f294e1c32a41fc94ff50e5c4b0b447e21549cbc642e21a31
  Stored in directory: /tmp/pip-ephem-wheel-cache-2wns173j/wheels/64/3d/9f/d12a217aa2531321c5b9ae96288fcae2687d3b744376e8f94f
Successfully built fastai
  Found existing installation: fastai 1.0.60
    Uninstalling fastai-1.0.60:
      Successfully uninstalled fastai-1.0.60
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [81.6 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x

Download pre-trained model

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
!wget "https://www.dropbox.com/s/iev3tkbz5wyyuz9/resnet101_caffe.pth"

--2020-01-10 09:01:47--  https://www.dropbox.com/s/iev3tkbz5wyyuz9/resnet101_caffe.pth
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/iev3tkbz5wyyuz9/resnet101_caffe.pth [following]
--2020-01-10 09:01:47--  https://www.dropbox.com/s/raw/iev3tkbz5wyyuz9/resnet101_caffe.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8ee9362efc3034091046f456e3.dl.dropboxusercontent.com/cd/0/inline/Av4cajzzQtBPsId1ULkc1sBoDT85HYKsZaGJtpaDycI5U5-RstMEbwIfuk_v7Xv39la49z58Il7FaA_A8d9RzPLvKhYJAR7aOObdIuYNQGXPTA/file# [following]
--2020-01-10 09:01:47--  https://uc8ee9362efc3034091046f456e3.dl.dropboxusercontent.com/cd/0/inline/Av4cajzzQtBPsId1ULkc1sBoDT85HYKsZaGJtpaDycI5U5-RstMEbwIfuk_v7Xv39la49z58Il7FaA_A8d9RzPLvKhYJAR7aOObdIuYNQGXPTA/

#Compilation

In [0]:
#!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [8]:
%cd faster-rcnn.pytorch/lib
!ls

/content/faster-rcnn.pytorch/lib
datasets  model  pycocotools  roi_data_layer  setup.py


In [9]:
!python setup.py build develop

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/datasets
copying datasets/coco.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/pascal_voc_rbg.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/ds_utils.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/vg_eval.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/imdb.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/imagenet.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/pascal_voc.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/__init__.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/factory.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/voc_eval.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/vg.py -> build/lib.linux-x86_64-3.6/datasets
creating build/lib.linux-x86_64-3.6/model
copying model/__init__.py -> build/lib.linux-x86_64-3.6/model
creating build/lib.linux-x86_

In [10]:
!pip uninstall scipy
!pip install scipy==1.1.0

Uninstalling scipy-1.4.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scipy-1.4.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/scipy/*
Proceed (y/n)? y
  Successfully uninstalled scipy-1.4.1
     |████████████████████████████████| 31.2MB 109kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


#Train

In [11]:
import os
os.chdir("/content/faster-rcnn.pytorch")
!pwd

/content/faster-rcnn.pytorch


# Change the folder name 'VOCdevkit' to 'VOCdevkit2007' in faster-rcnn.pytorch/data
# Make new folder and changed name to 'pretrained_model' in faster-rcnn.pytorch/data


In [0]:
# vgg 16
# !CUDA_VISIBLE_DEVICES=0 python lib/trainval_net.py \
#                    --dataset pascal_voc --net vgg16 \
#                    --bs 16 --nw 12 \
#                    --lr 0.001 --lr_decay_step 5 \
#                    --cuda

python3: can't open file 'trainval_net.py': [Errno 2] No such file or directory


In [15]:
# pascal_vod
!CUDA_VISIBLE_DEVICES=0 python trainval_net.py \
                   --dataset pascal_voc --net res101 \
                   --bs 4 --nw 12 \
                   --lr 0.001 --lr_decay_step 5 \
                   --cuda

Called with args:
Namespace(batch_size=4, checkepoch=1, checkpoint=0, checkpoint_interval=10000, checksession=1, class_agnostic=False, cuda=True, dataset='pascal_voc', disp_interval=100, large_scale=False, lr=0.001, lr_decay_gamma=0.1, lr_decay_step=5, mGPUs=False, max_epochs=20, net='res101', num_workers=12, optimizer='sgd', resume=False, save_dir='models', session=1, start_epoch=1, use_tfboard=False)
Using config:
{'ANCHOR_RATIOS': [0.5, 1, 2],
 'ANCHOR_SCALES': [8, 16, 32],
 'CROP_RESIZE_WITH_MAX_POOL': False,
 'CUDA': False,
 'DATA_DIR': '/content/faster-rcnn.pytorch/data',
 'DEDUP_BOXES': 0.0625,
 'EPS': 1e-14,
 'EXP_DIR': 'res101',
 'FEAT_STRIDE': [16],
 'GPU_ID': 0,
 'MATLAB': 'matlab',
 'MAX_NUM_GT_BOXES': 20,
 'MOBILENET': {'DEPTH_MULTIPLIER': 1.0,
               'FIXED_LAYERS': 5,
               'REGU_DEPTH': False,
               'WEIGHT_DECAY': 4e-05},
 'PIXEL_MEANS': array([[[102.9801, 115.9465, 122.7717]]]),
 'POOLING_MODE': 'align',
 'POOLING_SIZE': 7,
 'RESNET': {'FIXED

In [0]:
# python trainval_net.py --dataset pascal_voc --net vgg16 \
#                        --bs 24 --nw 8 \
#                        --lr $LEARNING_RATE --lr_decay_step $DECAY_STEP \
#                        --cuda --mGPUs

#Test

In [0]:
python demo.py --net res101 \
               --checksession 1 --checkepoch 2 --checkpoint 2504 \
               --cuda --load_dir /models